<a href="https://colab.research.google.com/github/Mekatebi/NMA_DL_2023_Project/blob/main/NMA_DL_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codes for NMA DL 2023 project
[Doc](https://docs.google.com/document/d/1Oh_WXq_B8Tlgt-A1j7EOfHFZrHP-iDAI1nQmS0hMD6w)

## RoBERTa

### Set-up environment

In [1]:
!pip install -q requests datasets transformers[torch] evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.8 MB/s eta 0:00:00


In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
import torch
import evaluate
import numpy as np
import requests
from torch import nn
import pandas as pd

In [3]:
from huggingface_hub import notebook_login

# notebook_login()

### Load dataset

In [4]:
# Define the remote file to retrieve
url = 'https://zenodo.org/record/2667859/files/500_Reddit_users_posts_labels.csv'
# Define the local filename to save data
local_file = '/content/Dataset.csv'
# Make http request for remote file data
data = requests.get(url)
# Save file data to local copy
with open(local_file, 'wb')as file:
  file.write(data.content)

In [5]:
# Edit the dataset

dataset = pd.read_csv(local_file)
dataset = dataset[dataset['Label'].isin(['Ideation', 'Attempt'])]
dataset.to_csv('/content/Modified_Dataset.csv')

In [6]:
dataset = load_dataset('csv', data_files='/content/Modified_Dataset.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'User', 'Post', 'Label'],
        num_rows: 216
    })
})

In [8]:
dataset['train'][21]

{'Unnamed: 0': 42,
 'User': 'user-42',
 'Post': "['You should walk in, put the phone/computer away and do it. Yes, Im afraid because as I said, whenever I try it just ends up as pointlessness. Im afraid to try anything, Im afraid to go through with anything, for Fear that it wont accomplish anything, and that if it does, the person that sees it, hears it, judges it; Whatever, will laugh or dismiss it for someone elses work thats better as per previous recurrences of such a thing. Or theyll patronize me and act like its really good, but just not good enough.', 'I cant help you, Im sorry, I feel hopeless. Its nice that you can relate though. Im such a screw up. Everything I know, everyone I know, they just do so much better than me. No one ever gets disappointed with them, but me, Im a horrid failure. I feel like Im always looked down upon no matter what I do, theres always something wrong with what Ive done compared to others. I want help, Ive been searching for help. Just dont know wha

### Preprocess

In [9]:
def not_none(example):
    return example['Post'] is not None

dataset = dataset.filter(not_none)

dataset_sampled = dataset['train'].train_test_split(test_size=0.01, seed=21)['train']

train_val_test = dataset_sampled.train_test_split(test_size=0.2, seed=21)
train_dataset = train_val_test['train']
test_val_dataset = train_val_test['test']

test_val_split = test_val_dataset.train_test_split(test_size=0.5, seed=21)
validation_dataset = test_val_split['train']
test_dataset = test_val_split['test']

columns_to_keep = ['Post', 'Label']

columns_to_remove = [col for col in dataset_sampled.column_names if col not in columns_to_keep]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

le = LabelEncoder()

le.fit(dataset_sampled['Label'])

def encode_labels(example):
    example['Label'] = le.transform([example['Label']])[0]
    return example

train_dataset = train_dataset.map(encode_labels)
validation_dataset = validation_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

Filter:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [10]:
labels= le.classes_

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

id2label

{0: 'Attempt', 1: 'Ideation'}

In [11]:
train_dataset

Dataset({
    features: ['Post', 'Label'],
    num_rows: 170
})

In [12]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def prepare_data(example):
    encoding = tokenizer.encode_plus(
        example['Post'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt',
    )
    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(example['Label'], dtype=torch.long)
    }

train_dataset = train_dataset.map(prepare_data)
validation_dataset = validation_dataset.map(prepare_data)
test_dataset = test_dataset.map(prepare_data)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [13]:
train_dataset

Dataset({
    features: ['Post', 'Label', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 170
})

### Evaluate

In [14]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Model

In [15]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                                         num_labels=len(le.classes_),
                                                         id2label=id2label,
                                                         label2id=label2id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Attempt",
    "1": "Ideation"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Attempt": 0,
    "Ideation": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

### Train

In [17]:
training_args = TrainingArguments(
    output_dir='./Model',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    push_to_hub=False,
    hub_model_id = "NMA_DL_2023_Project"
)

In [18]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([3.0, 1.0, 1.0, 1.0, 1.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [19]:
# CustomTrainer for Multiclass

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.605708,0.809524
2,No log,0.525906,0.809524
3,No log,0.508421,0.809524
4,No log,0.489315,0.809524
5,No log,0.491722,0.809524


TrainOutput(global_step=55, training_loss=0.5788989327170632, metrics={'train_runtime': 113.4591, 'train_samples_per_second': 7.492, 'train_steps_per_second': 0.485, 'total_flos': 223644397056000.0, 'train_loss': 0.5788989327170632, 'epoch': 5.0})

In [21]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.35285845398902893,
 'eval_accuracy': 0.9090909090909091,
 'eval_runtime': 0.7192,
 'eval_samples_per_second': 30.589,
 'eval_steps_per_second': 4.171,
 'epoch': 5.0}

In [22]:
# Push to Hugging Face

tokenizer.push_to_hub("NMA_DL_2023_Project")
model.push_to_hub("NMA_DL_2023_Project")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mekatebi/NMA_DL_2023_Project/commit/5cc1ed2ec2eb1f36deea250e02ff3ac628aa953a', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='5cc1ed2ec2eb1f36deea250e02ff3ac628aa953a', pr_url=None, pr_revision=None, pr_num=None)

## SHAP

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("Mekatebi/NMA_DL_2023_Project") # , revision="v2.0.0")
model = RobertaForSequenceClassification.from_pretrained("Mekatebi/NMA_DL_2023_Project") # , revision="v2.0.0")

In [41]:
!pip install -q shap

In [ ]:
import transformers
import shap

pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer,  return_all_scores=True)
explainer = shap.Explainer(pred)

In [ ]:
shap_values_example = explainer(test_dataset['Post'][21])

In [ ]:
shap.plots.text(shap_values_example)

## BertViz

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("Mekatebi/NMA_DL_2023_Project") # , revision="v2.0.0")
model = RobertaForSequenceClassification.from_pretrained("Mekatebi/NMA_DL_2023_Project") # , revision="v2.0.0")

In [ ]:
!pip install -q bertviz

### Neuron View

In [43]:
from bertviz.transformers_neuron_view import RobertaModel, RobertaTokenizer
from bertviz.neuron_view import show

In [ ]:
model_type = 'roberta'
model_version = 'roberta-base'
model = model
tokenizer = tokenizer
sentence_a = "The cat sat on the mat"
sentence_b = sentence_a
show(model, model_type, tokenizer, sentence_a, sentence_b)